while creating parquet tables you need to give schema its mandatory ... 
where as its not required for delta tables and while writing data to table 

<df.write.format('delta).mode('append').option("mergeSchema", "true").insertInto('mydb.trial_test')> 

give the option merge schema as true 


use query is describe extended <table name>

In [0]:
%sql
CREATE TABLE IF NOT EXISTS default.MyTable (
  key BINARY,
  value BINARY,
  topic STRING,
  partition INT,
  offset LONG,
  timestamp TIMESTAMP,
  timestampType INT
)
USING PARQUET

In [0]:
%sql
DESCRIBE EXTENDED mytable

col_name,data_type,comment
key,binary,null
value,binary,null
topic,string,null
partition,int,null
offset,bigint,null
timestamp,timestamp,null
timestampType,int,null
,,
# Detailed Table Information,,
Catalog,spark_catalog,


In [0]:
%sql
select * from default.mytable

key,value,topic,partition,offset,timestamp,timestampType


In [0]:
%sql
CREATE TABLE IF NOT EXISTS trial
USING DELTA

In [0]:
%sql
describe extended trial

col_name,data_type,comment
,,
# Detailed Table Information,,
Catalog,spark_catalog,
Database,default,
Table,trial,
Created Time,Mon Jan 29 11:06:03 UTC 2024,
Last Access,UNKNOWN,
Created By,Spark 3.3.2,
Type,MANAGED,
Location,dbfs:/user/hive/warehouse/trial,


In [0]:
dbutils.fs.ls('dbfs:/user/hive/warehouse/')

Out[21]: [FileInfo(path='dbfs:/user/hive/warehouse/mydb.db/', name='mydb.db/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/user/hive/warehouse/mytable/', name='mytable/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/user/hive/warehouse/sbit_db.db/', name='sbit_db.db/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/user/hive/warehouse/trial/', name='trial/', size=0, modificationTime=0)]

In [0]:
%sql
create database if not exists Mydb

In [0]:
%sql
CREATE TABLE IF NOT EXISTS mydb.trial_test
USING DELTA

In [0]:
%sql
CREATE TABLE IF NOT EXISTS mydb.um_table (
  key BINARY,
  value BINARY,
  topic STRING,
  partition INT,
  offset LONG,
  timestamp TIMESTAMP,
  timestampType INT
)
USING PARQUET
location '/umanaged'

In [0]:
kafka_server = "pkc-4r087.us-west2.gcp.confluent.cloud:9092"
kafka_api_secret = "SYRj4UPfB0qXJcYL32B+9FZYgT+SPsSlSyLA3Ww26v7NRxxLxBSdgP6A5Nqq842j"
base_dir_checkpoint = "/project_stuff/checkpoint/"
kafka_api_key = "PWEUCUXA4575GXRN"

df = (spark.read.format("kafka") 
     .option("kafka.bootstrap.servers", kafka_server) 
     .option("kafka.security.protocol", "SASL_SSL") 
     .option("kafka.sasl.jaas.config", "kafkashaded.org.apache.kafka.common.security.plain.PlainLoginModule required username='{}' password='{}';".format(kafka_api_key, kafka_api_secret)) 
     .option("kafka.ssl.endpoint.identification.algorithm", "https") 
     .option("kafka.sasl.mechanism", "PLAIN") 
     .option("subscribe","user_info_topic") 
     .load()
)




In [0]:
df.write.format('parquet').mode('overwrite').option("mergeSchema", "true").insertInto('mydb.um_table')

In [0]:
%sql
select count(*) from mydb.um_table

count(1)
13


In [0]:
%sql
describe extended mydb.um_table

col_name,data_type,comment
key,binary,null
value,binary,null
topic,string,null
partition,int,null
offset,bigint,null
timestamp,timestamp,null
timestampType,int,null
,,
# Detailed Table Information,,
Catalog,spark_catalog,


In [0]:
dbutils.fs.ls('/umanaged')

Out[46]: [FileInfo(path='dbfs:/umanaged/_SUCCESS', name='_SUCCESS', size=0, modificationTime=1706528400000),
 FileInfo(path='dbfs:/umanaged/_committed_5547533715140586513', name='_committed_5547533715140586513', size=324, modificationTime=1706528399000),
 FileInfo(path='dbfs:/umanaged/_started_5547533715140586513', name='_started_5547533715140586513', size=0, modificationTime=1706528335000),
 FileInfo(path='dbfs:/umanaged/part-00000-tid-5547533715140586513-aee1ef74-f00a-40c2-9f08-d415f135f7ff-124-1-c000.snappy.parquet', name='part-00000-tid-5547533715140586513-aee1ef74-f00a-40c2-9f08-d415f135f7ff-124-1-c000.snappy.parquet', size=934, modificationTime=1706528335000),
 FileInfo(path='dbfs:/umanaged/part-00001-tid-5547533715140586513-aee1ef74-f00a-40c2-9f08-d415f135f7ff-125-1-c000.snappy.parquet', name='part-00001-tid-5547533715140586513-aee1ef74-f00a-40c2-9f08-d415f135f7ff-125-1-c000.snappy.parquet', size=5673495, modificationTime=1706528379000),
 FileInfo(path='dbfs:/umanaged/part-00002